This script re-scales the Hansen data mask to the MODIS LAI resolution.

In [13]:
# setting up all the packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import torch
import torch.nn as nn

import rasterio
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject
import geopandas as gpd
from rasterio.mask import mask
import rioxarray as rxr

#from utils import * 

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

plt.rcParams['savefig.dpi'] = 400
plt.rcParams['font.size'] = 13
plt.rcParams["legend.frameon"] = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# call in the MODIS LAI product from the LEAP data catalogue
store = 'gs://leap-persistent/data-library/GLAB-CSIF/GLAB-CSIF.zarr'
ds = xr.open_dataset(store, engine='zarr', chunks={})

# get resolution of MODIS LAI data product
# get difference between lat and lon
res_lat = ds.lat.diff("lat").mean().item()
res_lon = ds.lon.diff("lon").mean().item()

print("Resolution (lat):", res_lat)
print("Resolution (lon):", res_lon)

/tmp/ipykernel_829/3053853174.py:3: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  ds = xr.open_dataset(store, engine='zarr', chunks={})


Resolution (lat): -0.050000000000001245
Resolution (lon): 0.05000000000001137


In [16]:
print(hansen)

<xarray.DataArray (band: 1, y: 86, x: 90)> Size: 8kB
[7740 values with dtype=uint8]
Coordinates:
  * band         (band) int64 8B 1
  * y            (y) float64 688B -9.812 -9.854 -9.896 ... -13.27 -13.31 -13.35
  * x            (x) float64 720B -72.4 -72.35 -72.31 ... -68.77 -68.73 -68.69
    spatial_ref  int64 8B 0
Attributes:
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    TIFFTAG_RESOLUTIONUNIT:  1 (unitless)
    AREA_OR_POINT:           Area
    _FillValue:              0
    scale_factor:            1.0
    add_offset:              0.0


In [10]:
# open the Hansen data mask dataset
hansen_datamask_path = "/home/jovyan/MLEAEEE4000-DroughtAmazon2/data/mdd_extent_TCscale.tif"   # Hansen data, raster to be resampled
hansen = rioxarray.open_rasterio(hansen_datamask_path)

In [26]:
# set raster path 
hansen_datamask_path = "/home/jovyan/MLEAEEE4000-DroughtAmazon2/data/mdd_extent_TCscale.tif"   # Hansen data, raster to be resampled

# Load raster
hansen = rxr.open_rasterio(hansen_datamask_path)

# set desired new resolution to the resolution of the MODIS LAI product, defined above
new_res = (res_lon, res_lat)

# resampling using "nearest" method, best for categorical data
hansen_resampled = hansen.rio.reproject(
    hansen.rio.crs,
    resolution=new_res,
    resampling="nearest"
)


In [ ]:
# export the resampled datamask
hansen_resampled.rio.to_raster("/home/jovyan/MLEAEEE4000-DroughtAmazon2/data/hansen_datamask.tif")